# Estimating Parameter Confidence Intervals With Bootstrapping
This notebook demonstrates the calculations required to do confidence interval constructions.
1. Construct a good model. This means checking that we get good $R^2$ values (or other model quality metrics) for each fold in a cross validation.
1. Compute residuals for the good model.
1. Construct a collection of parameter estimates. That is, for many repetitions
   1. Construct new observations (by using randomly selected residuals)
   1. Estimate parameter values
1. Compute the mean and standard deviation of the parameter estimates
1. Construct the confidence interval

In [1]:
%matplotlib inline
import tellurium as te
import numpy as np
import lmfit   # Fitting lib
import math
import random 
import matplotlib.pyplot as plt
import model_fitting as mf

In [2]:
# Model used in this example
model = """
     A -> B; k1*A
     B -> C; k2*B
      
     A = 5;
     B = 0;
     C = 0;
     k1 = 0.1
     k2 = 0.2
"""
unfitted_parameters = mf.makeParameters(constants=['k1', 'k2'])
unfitted_parameters

name,value,initial value,min,max,vary
k1,1.00000000,1,0.00000000,10.0000000,True
k2,1.00000000,1,0.00000000,10.0000000,True


In [3]:
# Globals
num_points = 10
sim_time = 30
nose_std = 0.5

In [4]:
# Create synthetic observational data for this example. This is for demonstration purposes only.
# In practice, you will have observational data from experiments.
obs_data = mf.makeObservations(model=model, noise_std=nose_std, num_points=num_points, sim_time=sim_time)
obs_data

       time,      [A],      [B],     [C]
 [[       0,  5.05862,        0,       0],
  [ 3.33333,  3.94984,   1.7743, 1.18526],
  [ 6.66667,  2.44081,   1.7099, 1.96265],
  [      10,  2.17076, 0.992598, 1.44905],
  [ 13.3333,  2.02888, 0.911974,  3.0343],
  [ 16.6667, 0.774877, 0.838443, 3.40189],
  [      20, 0.662598, 0.374512, 3.46384],
  [ 23.3333, 0.650324,        0,  4.3442],
  [ 26.6667,        0, 0.780101,  3.9221],
  [      30, 0.506843,  0.29932, 5.38783]]

## Step 1: Construct a good model.
In the following, we use the same model as the synthetic observations. Of course, in practice, you won't know the "true" model. You'll try many, and choose the best in terms of the quality metrics (e.g., $R^2$).

In [5]:
# Do the cross validation for this model. the crossValidate function returns two values: list of
# the parameters (for each fold) and RSQs for each fold.
list_parameters, rsqs = mf.crossValidate(obs_data, model=model, parameters=unfitted_parameters, 
                                         num_points=num_points, 
                                         sim_time=sim_time,
                                         num_folds=3)
rsqs

[0.9608181639684432, 0.9213467822481434, 0.8752391723959418]

These are very good $R^2$ values. So, we accept the model.

Next, we need to estimate the parameter values to use in our model. To this end, we do a fit on the full set of data.

In [6]:
fitted_parameters = mf.fit(obs_data, model=model, parameters=unfitted_parameters,
                           num_points=num_points, sim_time=sim_time)
fitted_parameters

name,value,standard error,relative error,initial value,min,max,vary
k1,0.09969554,0.00781533,(7.84%),1,0.00000000,10.0000000,True
k2,0.21261362,0.03237904,(15.23%),1,0.00000000,10.0000000,True


# Step 2: Compute the Residuals
Residuals need to be calculated by chemical species since they may be in very different units.

In [33]:
def makeResidualsBySpecies(obs_data, model, parameters, num_points, sim_time):
    """
    Calculate residuals for each chemical species.
    :param np.array obs_data: matrix of observations; first column is time; number of rows is num_points
    :param str model:
    :param lmfit.Parameters parameters.
    :param int num_points:
    :param int sim_time:
    :return np.array: matrix of residuals; columns are chemical species
    """
    data = mf.runSimulation(model=model, parameters=parameters, num_points=num_points, sim_time=sim_time)
    residuals = mf.calcSimulationResiduals(parameters, obs_data, num_points=num_points, sim_time=sim_time,
                                          model=model)
    # Reshape the residuals by species
    rr = te.loada(model)
    num_species = rr.getNumFloatingSpecies()
    nrows = int(len(residuals) / num_species)
    return np.reshape(residuals, (nrows, num_species))

In [8]:
# Note that the residuals for the chemical species differ. Compare the residuals for A (1st col) with
# the residuals for C (3rd col)
residuals_matrix = makeResidualsBySpecies(obs_data, model, fitted_parameters, num_points, sim_time)
residuals_matrix

 [[  0.0586228,          0,         0],
  [   0.363541,   0.781116,  0.764731],
  [  -0.131496,   0.508612,  0.736237],
  [   0.325759,  -0.109717, -0.603631],
  [    0.70553, 0.00284522,  0.266773],
  [  -0.174306,   0.128035,  0.061478],
  [ -0.0182113,  -0.163748, -0.317088],
  [   0.162007,  -0.400206,   0.23272],
  [   -0.35025,   0.486091, -0.433644],
  [   0.255624,  0.0850135,  0.853355]]

In [9]:
# The standard deviation of the residuals should be approximately the same as the standard deviation
# of the random noise we injected in the construction of the observations.
np.std(residuals_matrix)

0.38353289226237675

## Step 3: Construct a Collection of Parameter Estimates

### Step 3a: Construct Synthetic Observations
We define a function that constructs a set of observations from residuals and a model.

In [10]:
def makeSyntheticObservations(residual_matrix, model, parameters, num_points, sim_time):
    """
    Constructs synthetic observations for the model.
    :param np.array obs_data: matrix of residuals; columns are species; number of rows is num_points
    :param str model: Antimony model
    :param lmfit.Parameters parameters: parameters to use in the simulation
    :param int num_points:
    :param int sim_time:
    :return np.array: matrix; first column 
    """
    model_data = mf.runSimulation(model=model, parameters=fitted_parameters, 
                            num_points=num_points, sim_time=sim_time)
    data = model_data.copy()
    nrows, ncols = np.shape(data)
    for icol in range(1, ncols):  # Avoid the time column
        indices = np.random.randint(0, nrows, nrows)
        for irow in range(nrows):
            data[irow, icol] = max(data[irow, icol] + residuals_matrix[indices[irow], icol-1], 0)
    return data

In [11]:
# Try running this several times to see how values change.
makeSyntheticObservations(residuals_matrix, model, fitted_parameters, num_points, sim_time)

 [[       0,  5.36354, 0.00284522,        0],
  [ 3.33333,   3.7483,   0.592975, 0.482003],
  [ 6.66667,  2.82793,   0.801077,  1.96265],
  [      10,  2.00701,   0.992598,  1.44905],
  [ 13.3333,  1.64911,   0.745381,  2.76753],
  [ 16.6667, 0.930971,   0.310201,  4.07665],
  [      20, 0.739432,   0.666294,  4.01365],
  [ 23.3333, 0.546939,   0.528241,  3.50785],
  [ 26.6667, 0.605873,   0.379023,  4.03865],
  [      30,        0,   0.104589,  5.27071]]

### Step 3b: Repeatedly estimate parameter values

In [12]:
def makeParametersList(model, fitted_parameters, residuals_matrix, num_points, sim_time):
    list_parameters = []
    for _ in range(10):
        obs_data = makeSyntheticObservations(residuals_matrix, model, fitted_parameters, num_points, sim_time)
        parameters = mf.fit(obs_data, model=model, parameters=unfitted_parameters)
        list_parameters.append(parameters)
    return list_parameters

In [13]:
list_parameters = makeParametersList(model, fitted_parameters, residuals_matrix, num_points, sim_time)
list_parameters

[Parameters([('k1',
              <Parameter 'k1', value=0.10193296517802208 +/- 0.00751, bounds=[0:10]>),
             ('k2',
              <Parameter 'k2', value=0.23918098212748373 +/- 0.0369, bounds=[0:10]>)]),
 Parameters([('k1',
              <Parameter 'k1', value=0.10392072222272841 +/- 0.00814, bounds=[0:10]>),
             ('k2',
              <Parameter 'k2', value=0.26971766220705573 +/- 0.048, bounds=[0:10]>)]),
 Parameters([('k1',
              <Parameter 'k1', value=0.11078347518868714 +/- 0.00967, bounds=[0:10]>),
             ('k2',
              <Parameter 'k2', value=0.22069361482518757 +/- 0.0359, bounds=[0:10]>)]),
 Parameters([('k1',
              <Parameter 'k1', value=0.1027558132070211 +/- 0.00771, bounds=[0:10]>),
             ('k2',
              <Parameter 'k2', value=0.20403510089068988 +/- 0.0285, bounds=[0:10]>)]),
 Parameters([('k1',
              <Parameter 'k1', value=0.10381358721663736 +/- 0.008, bounds=[0:10]>),
             ('k2',
              <Pa

## Step 4: Compute the Mean and Standard Deviation of Parameters

In [14]:
def makeParameterStatistics(list_parameters):
    """
    Computes the mean and standard deviation of the parameters in a list of parameters.
    :param list-lmfit.Parameters
    :return dict: key is the parameter name; value is the tuple (mean, stddev)
    """
    parameter_statistics = {}  # This is a dictionary that will have the parameter name as key, and mean, std as values
    parameter_names = list(list_parameters[0].valuesdict().keys())
    for name in parameter_names:
        parameter_statistics[name] = []  # We will accumulate values in this list
        for parameters in list_parameters:
            parameter_statistics[name].append(parameters.valuesdict()[name])
    # Calculate the statistics
    for name in parameter_statistics.keys():
        mean = np.mean(parameter_statistics[name])
        std = np.std(parameter_statistics[name])
        std = std/np.sqrt(len(list_parameters))  # adjustments for the standard deviation of the mean
        parameter_statistics[name] = (mean, std)
    return parameter_statistics

In [15]:
# Here's the result
makeParameterStatistics(list_parameters)

{'k1': (0.10347987282768756, 0.0015147528067115995),
 'k2': (0.21978518974984915, 0.007803003492600655)}

## Step 5: Construct Confidence Intervals

# Exercises

## Exercise 1: Calculate confidence intervals.
1. Calculate the 95% confidence interval for 'k1' and 'k2' above.
1. Write a function that calculates the 95% confidence interval. Generalize it to any percentile.

## Exercise 2: Analyze a new model.

TRUE MODEL:

- A -> B
- A -> C
- B + C -> D

All kinetics are mass action. The kinetics constants are (in order of the reactions): 0.5, 0.5, 1.0. The initial concentration of A is 5. Consider a time course of duration 30 with 20 points.


1. Generate synthetic observations using this model using a normally distributed noise with a standard deviation
of 0.1.
1. Using the true model (the one above), find the $R^2$ values in a cross validation with 4 folds.
1. Construct confidence intervals for the parameters.